# Organize directories

The first step in training a ML model on sequences of images acquired from the ProtectCode app is to ensure the directory structure is set up for subsequent data loading. 

These commands assume the following: 

1) That the top directory for the training data and this notebook are both located in ~/src/YPB-AI/ on the EC2 instance. To get a zipped directory from S3 to that directory on your EC2 instance, you can run the following bash command: "aws s3 cp s3://your-bucket-name/yourZipFile.zip ~/src/YPB-AI/yourZipFile.zip". And to unzip it, you can run "unzip yourZipFile.zip"

2) That the jupyter notebook server was opened from the ~/src/ directory on the EC2 instance via the command "jupyter notebook --ip=0.0.0.0 --no-browser".

3) That underneath the top directory (which I usually call "Training" or something of the like), there are a series of subdirectories named and organized by classification label. For instance, each subdirectory that contains true negative image sequences, according to my implementation, contains the word "Negative". And subdirectories with true positives do not contain the word "Negative". 



In [1]:
import numpy as np
from glob import glob
import os, sys
import re

Set the name of your top data directory below. 

In [2]:
# topDirName = 'Training'
topDirName = 'Training2'

Lump together all subdirectories with the same classification label. 

In [3]:
#Assumes current working directory is ~/src/YPB-AI
allSamples = np.array(glob(topDirName + '/*'))
os.system('mkdir ' + topDirName + '/Absent')
os.system('mkdir ' + topDirName + '/Present')

for sample in allSamples: 
    if 'Negative' in sample or 'Sat' in sample:
        cmdString = "mv " + sample + " " + topDirName + "/Absent"
    else:
        cmdString = "mv " + sample + " " + topDirName + "/Present"
    os.system(cmdString)

Each directory within the classification subdirectory contains a series of folders that have the following contents:

1) A .txt file containing some timing and other metadata about the app run.

2) A folder called 'Photos' which contains the sequence of images acquired.

The next cell will obliterate the middle directory (the directory just below the classification directory) and repackage each run's photos into a uniquely identifiable folder of its own (lacking the text file) at the level where the middle directories once sat.



In [4]:
# Find 'vestigial' directories to be deleted after unpacking.
presentVestDirs = np.array(glob(topDirName + '/Present/*'))
absentVestDirs = np.array(glob(topDirName + '/Absent/*'))

# Unpack the JPG photo files into identifiable directories.
allPresentSamples = np.array(glob(topDirName + '/Present/*/*'))
for sample in allPresentSamples:
    # Grab the unique identifiers -- phone model, sample # and date/time
    digestedSample = re.split('/',sample)
    digestedTimestamp = re.split('_',digestedSample[3])
    newDir = digestedSample[2] + '_' + digestedTimestamp[1] + '_' + digestedTimestamp[2]
    # Make the new directory
    cmdString = 'mkdir ' + topDirName + '/Present/' + newDir
    os.system(cmdString)
    # Add the photos directly to it
    cmdString = 'mv ' + sample + '/Photos/*' + ' ' + topDirName + '/Present/' + newDir
    os.system(cmdString)
    
allAbsentSamples = np.array(glob(topDirName + '/Absent/*/*'))
for sample in allAbsentSamples:
    # Grab the unique identifiers -- phone model, sample # and date/time
    digestedSample = re.split('/',sample)
    digestedTimestamp = re.split('_',digestedSample[3])
    newDir = digestedSample[2] + '_' + digestedTimestamp[1] + '_' + digestedTimestamp[2]
    # Make the new directory
    cmdString = 'mkdir ' + topDirName + '/Absent/' + newDir
    os.system(cmdString)
    # Add the photos directly to it
    cmdString = 'mv ' + sample + '/Photos/*' + ' ' + topDirName + '/Absent/' + newDir
    os.system(cmdString)
    
    
# Delete vestigial directories.
for direc in presentVestDirs:
    os.system('rm -rf ' + direc)
    
for direc in absentVestDirs:
    os.system('rm -rf ' + direc)

Look through all directories and find image sequences containing the file "IMG_9999.JPG". If found, sort and rename to an appropriate image sequence. This is necessary because when iOS reaches "IMG_9999.JPG" it wraps back around to "IMG_0000.JPG" for the next in the sequence. If this is left as is, the sorting operation that follows will sort the images into the incorrect timing order. 

In [5]:
# Get the filenames.
imgSeqsToCorrect = os.popen('cd ~/src/YPB-AI/; find -name "*IMG_9999.JPG"').read()
# Split into unique entities.
imgSeqsToCorrect = re.split('\n',imgSeqsToCorrect)
# Drop the last entry, it's empty.
imgSeqsToCorrect.pop();

# Change any files that don't start with 'IMG_9' (i.e., any files that come after the wraparound) to start with 'J'.
# This will mean that when they are sorted, they will come correctly after the wraparound.
for sample in imgSeqsToCorrect:
    digestedSample = re.split('/',sample)
    pathToDir = os.path.join(digestedSample[0],digestedSample[1],digestedSample[2],digestedSample[3])
    filesInDir = os.listdir(pathToDir)
    for i in range(len(filesInDir)):
        if 'IMG_9' not in filesInDir[i]:
            newFile = filesInDir[i].replace('I','J',1)
            cmdString = 'mv ' + os.path.join(pathToDir,filesInDir[i]) + ' ' + os.path.join(pathToDir,newFile)
            os.system(cmdString)

# Change the images in the folder into a stack of 12-image input

In [5]:
import torch
import torchvision.models as models
from PIL import Image
import pickle

Use a pretrained 11-layer VGG with the final classifier layer chopped off.

In [7]:
# Import the CNN model and move it to CUDA if available.
model = models.vgg(pretrained=True)

for param in model.parameters():
    param.requires_grad = False
del model.classifier

if torch.cuda.is_available():
    model.to('cuda');

Define the function that will transform model output into a pickle file.

In [21]:
def transform_to_pickle(model,topdir):
    model.eval()
    ndx = 0
    topdirs = [direc for direc in os.listdir(topdir) if direc[0] is not '.']
    
    # Calculate the total number of stacks to transform.
    numStacks = 0
    for classDir in topdirs:
        thisClassDir = os.path.join(topdir,classDir)
        directories = [direc for direc in os.listdir(thisClassDir) if direc[0] is not '.']
        numStacks += len(directories)
    
    for classDir in topdirs:
        thisClassDir = os.path.join(topdir,classDir)
        directories = [direc for direc in os.listdir(thisClassDir) if direc[0] is not '.']
        
        
        # Do the transformation.
        for photoDir in directories:
            
            thisPhotoDir = os.path.join(thisClassDir, photoDir)
            ##pin pin fix##
            
#             print(thisPhotoDir)
#             print(os.path.isfile(photoDir))
#             continue
            
            if not os.path.isdir(thisPhotoDir):
                continue

            ## end ##
            files = [photofile for photofile in os.listdir(thisPhotoDir) if photofile.lower().endswith('jpg')]
            print("Transforming image stack {}".format(thisPhotoDir))
            files.sort()
            output = []
            # Make sure to appropriately transform the image size and normalize the RGB values, as the model expects. 
            for file in files:
                thisFile = os.path.join(thisPhotoDir,file)
                imgArr = np.float32(np.array(Image.open(thisFile).resize((224,224))).reshape([1,3,224,224]))
                imgArr[0][0] = ((imgArr[0][0] - 123.675)/58.395)
                imgArr[0][1] = ((imgArr[0][1] - 116.28)/57.12)
                imgArr[0][2] = ((imgArr[0][2] - 103.53)/57.375)
                if torch.cuda.is_available():
                    output.append(model(torch.cuda.FloatTensor(imgArr)))
                else:
                    output.append(model(torch.FloatTensor(imgArr)))

            FinalOutput = torch.FloatTensor(len(files),1,1024)
            for i in range(len(files)):
                FinalOutput[i] = output[i]
            pathStr = thisPhotoDir + '.pickle'
            with open(pathStr,'wb') as f:
                pickle.dump(FinalOutput,f)
                ndx += 1
                print("{}% complete.".format(ndx/numStacks*100))
    

Do the transformation

In [23]:
transform_to_pickle(model,topDirName)

Transforming image stack Training/Absent/6S_Negative_Sample3_Light_16-Apr-2019_15-59-47
0.004384042086804034% complete.
Transforming image stack Training/Absent/XS_Sat2_07-Jun-2019_20-13-37
0.008768084173608068% complete.
Transforming image stack Training/Absent/6_Negative_Sample3_Back_Light_16-Apr-2019_13-17-37
0.013152126260412098% complete.
Transforming image stack Training/Absent/SE_Negative_Dark_15-Mar-2019_15-37-50
0.017536168347216136% complete.
Transforming image stack Training/Absent/XS_Sat_17-Apr-2019_11-36-09
0.021920210434020166% complete.
Transforming image stack Training/Absent/6_Negative_Sample3_Back_Light_16-Apr-2019_14-11-36
0.026304252520824196% complete.
Transforming image stack Training/Absent/6_Negative_Dark_13-Mar-2019_13-01-20
0.030688294607628234% complete.
Transforming image stack Training/Absent/XS_Sat2_07-Jun-2019_16-46-00
0.03507233669443227% complete.
Transforming image stack Training/Absent/7_Negative_Dark_13-Mar-2019_16-21-25
0.0394563787812363% complete.

0.33318719859710655% complete.
Transforming image stack Training/Absent/XS_Negative_Dark_14-Mar-2019_18-38-24
0.3375712406839106% complete.
Transforming image stack Training/Absent/XS_Sat2_07-Jun-2019_20-38-00
0.34195528277071463% complete.
Transforming image stack Training/Absent/XS_Sat2_07-Jun-2019_19-05-52
0.3463393248575186% complete.
Transforming image stack Training/Absent/XS_Sat2_07-Jun-2019_17-34-45
0.35072336694432266% complete.
Transforming image stack Training/Absent/SE_Negative_Random_03-Jun-2019_16-46-34
0.3551074090311267% complete.
Transforming image stack Training/Absent/XS_Negative_Dark_14-Mar-2019_18-15-22
0.35949145111793074% complete.
Transforming image stack Training/Absent/6S_Negative_Random_03-Jun-2019_14-42-51
0.3638754932047348% complete.
Transforming image stack Training/Absent/6_Negative_Dark_13-Mar-2019_11-41-49
0.3682595352915388% complete.
Transforming image stack Training/Absent/7_Negative_Random_30-May-2019_17-23-41
0.3726435773783428% complete.
Transfor

0.6663743971942131% complete.
Transforming image stack Training/Absent/XS_Negative_Random_04-Jun-2019_10-49-09
0.6707584392810171% complete.
Transforming image stack Training/Absent/6_Negative_Dark_13-Mar-2019_13-48-21
0.6751424813678212% complete.
Transforming image stack Training/Absent/6S_Negative_Sample3_Light_16-Apr-2019_15-52-30
0.6795265234546252% complete.
Transforming image stack Training/Absent/XS_Negative_Sample3_Light_17-Apr-2019_11-18-22
0.6839105655414293% complete.
Transforming image stack Training/Absent/6_Negative_Sample3_Back_Light_16-Apr-2019_13-21-27
0.6882946076282332% complete.
Transforming image stack Training/Absent/6_Negative_Dark_13-Mar-2019_12-48-00
0.6926786497150372% complete.
Transforming image stack Training/Absent/XS_Negative_Random_30-May-2019_14-45-16
0.6970626918018413% complete.
Transforming image stack Training/Absent/7_Negative_Random_30-May-2019_16-55-34
0.7014467338886453% complete.
Transforming image stack Training/Absent/SE_Negative_Dark_15-Mar

0.9995615957913196% complete.
Transforming image stack Training/Absent/XS_Sat2_07-Jun-2019_20-42-21
1.0039456378781235% complete.
Transforming image stack Training/Absent/6S_Negative_Dark_14-Mar-2019_11-33-51
1.0083296799649275% complete.
Transforming image stack Training/Absent/XS_Sat2_07-Jun-2019_19-11-48
1.0127137220517317% complete.
Transforming image stack Training/Absent/XS_Sat2_07-Jun-2019_19-31-47
1.0170977641385357% complete.
Transforming image stack Training/Absent/6S_Negative_Random_03-Jun-2019_14-41-23
1.0214818062253397% complete.
Transforming image stack Training/Absent/XS_Sat2_07-Jun-2019_20-52-31
1.0258658483121437% complete.
Transforming image stack Training/Absent/XS_Negative_Random_04-Jun-2019_10-31-17
1.0302498903989479% complete.
Transforming image stack Training/Absent/6S_Negative_Random_03-Jun-2019_14-49-27
1.0346339324857519% complete.
Transforming image stack Training/Absent/SE_Negative_Dark_15-Mar-2019_16-32-27
1.0390179745725558% complete.
Transforming image 

1.3415168785620342% complete.
Transforming image stack Training/Absent/7_Negative_Dark_13-Mar-2019_15-39-59
1.3459009206488384% complete.
Transforming image stack Training/Absent/7_Negative_Sample3_Light_16-Apr-2019_17-22-38
1.3502849627356424% complete.
Transforming image stack Training/Absent/7_Negative_Random_30-May-2019_16-23-35
1.3546690048224463% complete.
Transforming image stack Training/Absent/XS_Sat2_07-Jun-2019_19-07-57
1.3590530469092503% complete.
Transforming image stack Training/Absent/XS_Sat2_07-Jun-2019_17-09-04
1.3634370889960545% complete.
Transforming image stack Training/Absent/7_Negative_Dark_13-Mar-2019_16-27-46
1.3678211310828585% complete.
Transforming image stack Training/Absent/7_Negative_Random_30-May-2019_16-23-01
1.3722051731696625% complete.
Transforming image stack Training/Absent/XS_Sat_07-Jun-2019_14-01-54
1.3765892152564665% complete.
Transforming image stack Training/Absent/XS_Sat2_07-Jun-2019_19-37-51
1.3809732573432705% complete.
Transforming image

...

16.085050416484% complete.
Transforming image stack Training/Absent/7_Negative_Sample3_Light_16-Apr-2019_18-32-35
16.089434458570803% complete.
Transforming image stack Training/Absent/7_Negative_Dark_13-Mar-2019_17-35-50
16.093818500657605% complete.
Transforming image stack Training/Absent/XS_Negative_Dark_14-Mar-2019_17-22-14
16.09820254274441% complete.
Transforming image stack Training/Absent/XS_Sat_07-Jun-2019_14-01-02
16.102586584831215% complete.
Transforming image stack Training/Absent/XS_Sat2_07-Jun-2019_19-03-01
16.10697062691802% complete.
Transforming image stack Training/Absent/6_Negative_Dark_13-Mar-2019_14-16-39
16.111354669004825% complete.
Transforming image stack Training/Absent/XS_Negative_Dark_14-Mar-2019_19-20-37
16.115738711091627% complete.
Transforming image stack Training/Absent/XS_Sat2_07-Jun-2019_17-09-16
16.120122753178432% complete.
Transforming image stack Training/Absent/XS_Sat2_07-Jun-2019_17-02-56
16.124506795265233% complete.
Transforming image stack 

16.42262165716791% complete.
Transforming image stack Training/Absent/XS_Negative_Random_04-Jun-2019_10-09-30
16.427005699254714% complete.
Transforming image stack Training/Absent/XS_Sat2_07-Jun-2019_17-01-49
16.43138974134152% complete.
Transforming image stack Training/Absent/XS_Sat2_07-Jun-2019_16-48-48
16.43577378342832% complete.
Transforming image stack Training/Absent/XS_Sat_07-Jun-2019_14-27-19
16.440157825515126% complete.
Transforming image stack Training/Absent/6S_Negative_Dark_14-Mar-2019_10-12-59
16.444541867601927% complete.
Transforming image stack Training/Absent/7_Negative_Random_30-May-2019_17-11-07
16.448925909688732% complete.
Transforming image stack Training/Absent/7_Negative_Dark_13-Mar-2019_16-57-33
16.453309951775537% complete.
Transforming image stack Training/Absent/7_Negative_Dark_13-Mar-2019_16-54-35
16.45769399386234% complete.
Transforming image stack Training/Absent/6S_Negative_Dark_14-Mar-2019_10-42-51
16.462078035949148% complete.
Transforming image s

16.755808855765014% complete.
Transforming image stack Training/Absent/SE_Negative_Dark_15-Mar-2019_15-40-15
16.76019289785182% complete.
Transforming image stack Training/Absent/XS_Negative_Sample3_Light_17-Apr-2019_12-21-56
16.764576939938625% complete.
Transforming image stack Training/Absent/7_Negative_Dark_13-Mar-2019_16-18-05
16.768960982025426% complete.
Transforming image stack Training/Absent/XS_Sat2_07-Jun-2019_18-04-13
16.77334502411223% complete.
Transforming image stack Training/Absent/XS_Negative_Dark_14-Mar-2019_17-41-50
16.777729066199036% complete.
Transforming image stack Training/Absent/XS_Sat_07-Jun-2019_13-48-03
16.78211310828584% complete.
Transforming image stack Training/Absent/XS_Sat2_07-Jun-2019_18-06-27
16.786497150372643% complete.
Transforming image stack Training/Absent/6S_Negative_Random_03-Jun-2019_14-23-02
16.790881192459448% complete.
Transforming image stack Training/Absent/XS_Sat2_07-Jun-2019_16-49-35
16.795265234546253% complete.
Transforming image 

17.08899605436212% complete.
Transforming image stack Training/Absent/SE_Negative_Dark_15-Mar-2019_15-43-29
17.093380096448925% complete.
Transforming image stack Training/Absent/XS_Negative_Dark_14-Mar-2019_17-33-28
17.09776413853573% complete.
Transforming image stack Training/Absent/6_Negative_Dark_13-Mar-2019_11-41-03
17.102148180622535% complete.
Transforming image stack Training/Absent/7_Negative_Dark_13-Mar-2019_17-16-11
17.10653222270934% complete.
Transforming image stack Training/Absent/XS_Sat2_07-Jun-2019_19-25-16
17.110916264796142% complete.
Transforming image stack Training/Absent/6_Negative_Dark_13-Mar-2019_12-38-10
17.115300306882947% complete.
Transforming image stack Training/Absent/7_Negative_Random_30-May-2019_16-21-33
17.11968434896975% complete.
Transforming image stack Training/Absent/XS_Sat2_07-Jun-2019_19-23-37
17.124068391056554% complete.
Transforming image stack Training/Absent/XS_Sat2_07-Jun-2019_17-28-17
17.128452433143355% complete.
Transforming image sta

KeyboardInterrupt: 

Clean up the directory, removing everything but the pickle files.

In [24]:
# Assemble all the pickle files and remove the rest.
classDirs = [direc for direc in os.listdir(topDirName) if direc[0] is not '.']
for classDir in classDirs:
    newDir = os.path.join(topDirName,classDir)
    fileDirs = [direc for direc in os.listdir(newDir) if direc[0] is not '.' and not direc.lower().endswith('.pickle')]
    for fileDir in fileDirs:
        pickleFileDir = os.path.join(newDir,fileDir)
        os.system('rm -rf ' + pickleFileDir)

# Split the directory into train/test/validation sets

In [25]:
import random

In [26]:
finalDirName = 'Split_Data'

In [27]:
def split_folders(inputDir, outputDir, trainValTestRatio, seed):
    # Assume inputDir is organized as 'inputDir/class/topFolder/photos'
    classes = os.listdir(inputDir)
    # Exclude folders where '.' is the first character.
    classes = [directory for directory in classes if directory[0] != '.']
    numClasses = len(classes)
    # Create output directories
    os.system('mkdir ' + outputDir)
    os.system('mkdir ' + outputDir + '/train')
    os.system('mkdir ' + outputDir + '/valid')
    os.system('mkdir ' + outputDir + '/test')
    # Pull files and put into the appropriate folders.
    for directory in classes:
        files = os.listdir(os.path.join(inputDir, directory))
        files.sort()
        random.seed(seed)
        random.shuffle(files)
        numFiles = len(files)
        # Get the indices.
        ndxAmts = np.intc((np.round(trainValTestRatio / np.sum(trainValTestRatio) * numFiles)))
        numNdxes = np.sum(ndxAmts)
        if numNdxes != numFiles:
            ndxAmts[0] += numFiles-numNdxes
        trainFiles = files[0:ndxAmts[0]]
        valFiles = files[ndxAmts[0]:ndxAmts[0]+ndxAmts[1]]
        testFiles = files[ndxAmts[0]+ndxAmts[1]:]
        # Put the training directories in the appropriate folder
        # First make the class directories.
        os.system('mkdir ' + outputDir + '/train/' + directory)
        os.system('mkdir ' + outputDir + '/valid/' + directory)
        os.system('mkdir ' + outputDir + '/test/' + directory)
        for file in trainFiles:
            os.system('cp -r ' + inputDir + '/' + directory + '/' + file + ' ' + outputDir + '/train/' + directory + '/' + file)
        for file in valFiles:
            os.system('cp -r ' + inputDir + '/' + directory + '/' + file + ' ' + outputDir + '/valid/' + directory + '/' + file)
        for file in testFiles:
            os.system('cp -r ' + inputDir + '/' + directory + '/' + file + ' ' + outputDir + '/test/' + directory + '/' + file)

In [28]:
split_folders(topDirName, finalDirName,[0.8,0.1,0.1],7778)